In [ ]:
!pip install datasets

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 480.6/480.6 kB 9.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 6.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 179.3/179.3 kB 10.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 7.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 10.6 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2024.10.0
    Uninstalling fsspec-2024.10.0:
      Successfully uninstalled fsspec-2024.10.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gcsfs 2024.10.0 requires fsspec==2024.10.0, but you have fsspec 2024.9.0 which is incompatible.


In [ ]:
!pip install openai
!pip install evaluate

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.5/76.5 kB 4.3 MB/s eta 0:00:00
  Attempting uninstall: openai
    Found existing installation: openai 1.54.5
    Uninstalling openai-1.54.5:
      Successfully uninstalled openai-1.54.5
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.0/84.0 kB 2.6 MB/s eta 0:00:00


In [ ]:
!pip install rouge_score

  Preparing metadata (setup.py) ... done
  Created wheel for rouge_score: filename=rouge_score-0.1.2-py3-none-any.whl size=24935 sha256=350c9ec40e71604fc570e732dc57a46981f43497c5377ecb69ac058f8f11c627
  Stored in directory: /root/.cache/pip/wheels/5f/dd/89/461065a73be61a532ff8599a28e9beef17985c9e9c31e541b4
Successfully built rouge_score


In [ ]:
!pip install git+https://github.com/google-research/bleurt.git

  Cloning https://github.com/google-research/bleurt.git to /tmp/pip-req-build-7hovucly
  Running command git clone --filter=blob:none --quiet https://github.com/google-research/bleurt.git /tmp/pip-req-build-7hovucly
  Resolved https://github.com/google-research/bleurt.git to commit cebe7e6f996b40910cfaa520a63db47807e3bf5c
  Preparing metadata (setup.py) ... done
  Created wheel for BLEURT: filename=BLEURT-0.0.2-py3-none-any.whl size=16456764 sha256=18d84674198a7b6476bc1fd0a9997102f4b758706487b889f9fb66d33b5a6883
  Stored in directory: /tmp/pip-ephem-wheel-cache-eio9vrwu/wheels/64/f4/2c/509a6c31b8ebde891a81029fd94f199b1b92f0e7cfc20d417a
Successfully built BLEURT


In [ ]:
!pip install bert_score

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.1/61.1 kB 2.5 MB/s eta 0:00:00


In [ ]:
import openai
from datasets import load_dataset
from evaluate import load

In [ ]:
# Set up OpenAI API key
openai.api_key = ""
model_choice = "gpt-3.5-turbo"

In [ ]:
# Load the TellMeWhy dataset
ds = load_dataset("StonyBrookNLP/tellmewhy")

# Extract the first 1000 test samples
test_data = ds['test'][:1000]


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


README.md:   0%|          | 0.00/7.76k [00:00<?, ?B/s]

dataset_infos.json:   0%|          | 0.00/1.89k [00:00<?, ?B/s]

train.json:   0%|          | 0.00/70.1M [00:00<?, ?B/s]

validation.json:   0%|          | 0.00/8.71M [00:00<?, ?B/s]

test.json:   0%|          | 0.00/10.4M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/71892 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/8976 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/10689 [00:00<?, ? examples/s]

In [ ]:
# Load evaluation metrics
bleu = load("bleu")
rouge = load("rouge")
bleurt = load("bleurt")
bertscore = load("bertscore")

In [ ]:

def zero_shot_gpt3(narrative, question, model="gpt-3.5-turbo"):
    try:
        response = openai.ChatCompletion.create(
            model=model,
            messages=[
                {"role": "system", "content": "You are a helpful assistant for answering questions."},
                {"role": "user", "content": f"Context: {narrative}\nQuestion: {question}\nAnswer:"}
            ]
        )
        return response['choices'][0]['message']['content'].strip()
    except Exception as e:
        return f"An error occurred: {e}"


In [ ]:
from tqdm import tqdm

predictions = []
references = []
detailed_results = []

# Wrap the loop with tqdm for a progress bar
for idx in tqdm(range(len(test_data['narrative'])), desc="Processing Examples"):
    narrative = test_data['narrative'][idx]
    question = test_data['question'][idx]
    true_answer = test_data['answer'][idx]

    # Get GPT's zero-shot answer
    zero_shot_answer = zero_shot_gpt3(narrative, question)

    # Store the results
    predictions.append(zero_shot_answer)
    references.append([true_answer])  # BLEU expects a list of references

    detailed_results.append({
        "narrative": narrative,
        "question": question,
        "true_answer": true_answer,
        "predicted_answer": zero_shot_answer
    })


Processing Examples: 100%|██████████| 1000/1000 [09:31<00:00,  1.75it/s]


In [ ]:
print("Sample Predictions:")
for i, result in enumerate(detailed_results[:10]):
    print(f"Example {i + 1}:")
    print(f"  Narrative: {result['narrative']}")
    print(f"  Question: {result['question']}")
    print(f"  True Answer: {result['true_answer']}")
    print(f"  Predicted Answer: {result['predicted_answer']}\n")

Sample Predictions:
Example 1:
  Narrative: Bob was a computer scientist. He enjoyed natural language processing. He decided to revolutionize the industry! He formulated a machine learning algorithm to process words. He won the nobel prize for his work!
  Question: Why was Bob a computer scientist?
  True Answer: Bob enjoyed the language processing that was part of the job.
  Predicted Answer: Bob was a computer scientist because he had a passion for natural language processing and decided to revolutionize the industry by formulating a machine learning algorithm to process words. His expertise and achievements in this field led to him being recognized and awarded the Nobel Prize for his work.

Example 2:
  Narrative: Bob was a computer scientist. He enjoyed natural language processing. He decided to revolutionize the industry! He formulated a machine learning algorithm to process words. He won the nobel prize for his work!
  Question: Why did He enjoy natural language processing?
  Tru

In [ ]:
# Evaluate performance using metrics
bleu_score = bleu.compute(predictions=predictions, references=references)
rouge_scores = rouge.compute(predictions=predictions, references=[ref[0] for ref in references])
bleurt_scores = bleurt.compute(predictions=predictions, references=[ref[0] for ref in references])
bertscore_scores = bertscore.compute(predictions=predictions, references=[ref[0] for ref in references], lang="en")

# Display results
print("Performance Metrics:")
print(f"BLEU: {bleu_score['bleu']:.4f}")
print(f"ROUGE-L: {rouge_scores['rougeL']:.4f}")
print(f"BLEURT: {sum(bleurt_scores['scores']) / len(bleurt_scores['scores']):.4f}")
print(f"BERTScore (F1): {sum(bertscore_scores['f1']) / len(bertscore_scores['f1']):.4f}")

tokenizer_config.json:   0%|          | 0.00/25.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/482 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/899k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/1.42G [00:00<?, ?B/s]

Some weights of RobertaModel were not initialized from the model checkpoint at roberta-large and are newly initialized: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Performance Metrics:
BLEU: 0.0278
ROUGE-L: 0.1740
BLEURT: -0.8195
BERTScore (F1): 0.8840


In [ ]:
import random

# Pool of few-shot examples
few_shot_pool = [
    {
        "narrative": "Rudy was convinced that bottled waters all tasted the same. He went to the store and bought several popular brands. He went back home and set them all on a table. He spent several hours tasting them one by one. He came to the conclusion that they actually did taste different.",
        "question": "Why did Rudy go to the store?",
        "answer": "Rudy wanted to test the taste of bottled water."
    },
    {
        "narrative": "Sandra got a job at the zoo. She loved coming to work and seeing all of the animals. Sandra went to look at the polar bears during her lunch break. She watched them eat fish and jump in and out of the water. She took pictures and shared them with her friends.",
        "question": "Why did Sandra go to look at the polar bears?",
        "answer": "Sandra wanted to take pictures of the polar bears."
    },
    {
        "narrative": "Alex bought a new bicycle. He rode it every day to work. One day, he noticed a strange noise coming from the bike. He stopped to check the tires and found one was flat.",
        "question": "Why did Alex buy a new bicycle?",
        "answer": "Alex needed a way to commute to work."
    }
]


In [ ]:

# Function to select random examples
def select_random_examples(pool, num_examples=2):
    return random.sample(pool, num_examples)

# Few-shot GPT function with dynamic example selection
def few_shot_gpt3(narrative, question, pool, model="gpt-3.5-turbo"):
    examples = select_random_examples(pool)
    few_shot_prompt = "\n\n".join(
        f"Context: {ex['narrative']}\nQuestion: {ex['question']}\nAnswer: {ex['answer']}"
        for ex in examples
    )
    prompt = (
        few_shot_prompt
        + f"\n\nContext: {narrative}\nQuestion: {question}\nAnswer:"
    )
    try:
        response = openai.ChatCompletion.create(
            model=model,
            messages=[
                {"role": "system", "content": "You are a helpful assistant for answering questions."},
                {"role": "user", "content": prompt}
            ]
        )
        return response['choices'][0]['message']['content'].strip()
    except Exception as e:
        return f"An error occurred: {e}"


In [ ]:
# Collect predictions and references for few-shot
few_shot_predictions = []
few_shot_references = []
few_shot_detailed_results = []

# Run few-shot evaluation with progress tracking
for idx in tqdm(range(len(test_data['narrative'])), desc="Processing Few-Shot Examples"):
    narrative = test_data['narrative'][idx]
    question = test_data['question'][idx]
    true_answer = test_data['answer'][idx]

    random_examples = select_random_examples(few_shot_pool, num_examples=2)
    # Get GPT's few-shot answer
    few_shot_answer = few_shot_gpt3(narrative, question, random_examples)

    # Store results
    few_shot_predictions.append(few_shot_answer)
    few_shot_references.append([true_answer])  # BLEU expects a list of references
    few_shot_detailed_results.append({
        "narrative": narrative,
        "question": question,
        "true_answer": true_answer,
        "predicted_answer": few_shot_answer
    })

Processing Few-Shot Examples: 100%|██████████| 1000/1000 [07:20<00:00,  2.27it/s]


In [ ]:
# Evaluate performance
few_shot_bleu_score = bleu.compute(predictions=few_shot_predictions, references=few_shot_references)
few_shot_rouge_scores = rouge.compute(predictions=few_shot_predictions, references=[ref[0] for ref in few_shot_references])
few_shot_bleurt_scores = bleurt.compute(predictions=few_shot_predictions, references=[ref[0] for ref in few_shot_references])
few_shot_bertscore_scores = bertscore.compute(predictions=few_shot_predictions, references=[ref[0] for ref in few_shot_references], lang="en")

# Display results
print("Few-Shot Performance Metrics:")
print(f"BLEU: {few_shot_bleu_score['bleu']:.4f}")
print(f"ROUGE-L: {few_shot_rouge_scores['rougeL']:.4f}")
print(f"BLEURT: {sum(few_shot_bleurt_scores['scores']) / len(few_shot_bleurt_scores['scores']):.4f}")
print(f"BERTScore (F1): {sum(few_shot_bertscore_scores['f1']) / len(few_shot_bertscore_scores['f1']):.4f}")